- Chosen file format inputs
    - json
    - csv
    - dictionary
    - numpy array
    - list of tuples

**Target** => pandas dataframe

# Example under consideration
| Event(**key**) | Mass |
| --- | --- |
| No one is guilty(**phi**) | 0 |
| B is guilty(**B**) | 0.1 |
| J is guilty(**J**) | 0.2 |
| S is guilty(**S**) | 0.1 |
| B or J are guilty(**BJ**) | 0.1 |
| S or J are guilty(**SJ**) | 0.3 |
| B or S are guilty(**BS**) | 0.1 |
| One of the 3 guilty(**BJS**) | 0.1 |

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
# define global df
global df

In [15]:
# from_json helper
with open("./sample_data1/sample_data1.json", "r") as file:
    data_dict = json.load(file)
#     indices = data_dict.keys()
#     values = list(data_dict.values())
    df_inner = pd.DataFrame.from_dict(data_dict, columns=["Mass"], orient="index")
    df = df_inner
    df_inner.to_csv("./sample_data1/sample_data1.csv", encoding="utf-8")
    df_dict = df_inner.to_dict()
    df_list_of_tuples = df_inner.to_records()
    df_numpy_array = np.array(df_inner.to_records().view(type=np.matrix))
    display(df_inner)

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
SJ,0.3
BS,0.1
BJS,0.1


In [51]:
def from_json_helper(json_file_name):
    with open(json_file_name, "r") as file:
        data_dict = json.load(file)
        return_df = pd.DataFrame.from_dict(data_dict, columns=["Mass"], orient="index")
    return return_df

In [12]:
# from_dict helper
dict_input = df.to_dict()["Mass"]
print(dict_input)
return_df = pd.DataFrame.from_dict(dict_input, orient="index", columns=["Mass"])
display(return_df)

{'phi': 0.0, 'B': 0.1, 'J': 0.2, 'S': 0.1, 'BJ': 0.1, 'SJ': 0.3, 'BS': 0.1, 'BJS': 0.1}


,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
SJ,0.3
BS,0.1
BJS,0.1


In [39]:
def from_dict_helper(input_dict):
    return_df = pd.DataFrame.from_dict(input_dict, orient="index", columns=["Mass"])
    return return_df

In [37]:
# from_list_of_tuples helper
list_of_tuples_input = df.to_records()
print(dict(list_of_tuples_input))
return_list_of_tuples_df = pd.DataFrame.from_dict(dict(list_of_tuples_input), columns=["Mass"], orient="index")
display(return_list_of_tuples_df)

{'phi': 0.0, 'B': 0.1, 'J': 0.2, 'S': 0.1, 'BJ': 0.1, 'SJ': 0.3, 'BS': 0.1, 'BJS': 0.1}


,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
SJ,0.3
BS,0.1
BJS,0.1


In [40]:
def from_list_of_tuples_helper(list_of_tuples):
    return_list_of_tuples_df = pd.DataFrame.from_dict(dict(list_of_tuples_input), columns=["Mass"], orient="index")
    return return_list_of_tuples_df

In [29]:
# from numpy_array helper
np_array_input = np.array(df.to_records().view(type=np.matrix))
return_np_array_df = pd.DataFrame(np_array_input[0], columns=["Mass"], index=[i[0] for i in np_array_input[0]])
display(return_np_array_df)

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
SJ,0.3
BS,0.1
BJS,0.1


In [41]:
def from_numpy_array_helper(np_array_input):
    return_np_array_df = pd.DataFrame(np_array_input[0], columns=["Mass"], index=[i[0] for i in np_array_input[0]])
    return return_np_array_df

In [50]:
# from_csv helper
return_csv_df = pd.read_csv("./sample_data1/sample_data1.csv", index_col=0, header=None, names=["Mass"])
display(return_csv_df)

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
SJ,0.3
BS,0.1
BJS,0.1


In [52]:
def from_csv_helper(csv_file_name):
    return_csv_df = pd.read_csv("./sample_data1/sample_data1.csv", index_col=0, header=None, names=["Mass"])
    return return_csv_df

In [248]:
from collections import defaultdict
from itertools import combinations

class MassFunction:
    def __init__(self, input_var):
        if type(input_var) == str:
            ext = input_var.split(".")[-1]
            if ext == "json":
                self.frame_of_discernment = from_json_helper(input_var)
            elif ext == "csv":
                self.frame_of_discernment = from_csv_helper(input_var)
        else:
            if type(input_var) == dict:
                self.frame_of_discernment = from_dict_helper(input_var)
            elif type(input_var) == list:
                self.frame_of_discernment = from_list_of_tuples_helper(input_var)
            elif type(input_var) == np.ndarray:
                self.frame_of_discernment = from_numpy_array_helper(input_var)
                
        assert self.frame_of_discernment is not None
        self.frame_of_discernment.index = ["phi"] + ["".join(sorted(id)) for id in list(self.frame_of_discernment.index[1:])]
        self.compute_metrics()
        
    def compute_metrics(self):
        self.frame_of_discernment_indices_without_phi = list(self.frame_of_discernment.index[1:])
        self.intersection_dict = defaultdict(list)
        for a, b in list(combinations(self.frame_of_discernment_indices_without_phi, 2)):
            if set(a).intersection(set(b)):
                self.intersection_dict[a].append(b)
                self.intersection_dict[b].append(a)
        for key in self.intersection_dict.keys():
            self.intersection_dict[key].append(key)
        self.compute_belief()
        self.compute_plausability_and_disbelief()
        
    def compute_belief(self):
        self.belief = {}
        focal_elements_ = []
        for a in self.frame_of_discernment_indices_without_phi:
            focal_elements_ = []
            for i in range(1, len(a) + 1):
                focal_elements_.extend(["".join(c) for c in list(combinations(a, i))])
            self.belief.update({a:self.frame_of_discernment.reindex(focal_elements_).sum()["Mass"]})
        
        assert self.belief is not None
        
    def compute_plausability_and_disbelief(self):
        self.plausibility = {}
        self.disbelief = {}
        for a in self.intersection_dict.keys():
            plausability_value = self.frame_of_discernment.reindex(self.intersection_dict[a]).sum()["Mass"]
            self.plausibility.update({a : plausability_value})
            self.disbelief.update({a : 1 - plausability_value})
    
    def get_belief(self):
        return self.belief
        
    def get_plausability(self):
        return self.plausibility
    
    def get_disbelief(self):
        return self.disbelief
    
    def __repr__(self):
        display(self.frame_of_discernment)

In [249]:
from itertools import  combinations
from collections import defaultdict

In [250]:
a = MassFunction("./sample_data1/sample_data1.json")

In [251]:
a.frame_of_discernment

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
JS,0.3
BS,0.1
BJS,0.1


In [252]:
a.get_belief()

{'B': 0.1,
 'J': 0.2,
 'S': 0.1,
 'BJ': 0.4,
 'JS': 0.6000000000000001,
 'BS': 0.30000000000000004,
 'BJS': 0.9999999999999999}

In [253]:
a.get_plausability()

{'B': 0.4,
 'BJ': 0.9,
 'BS': 0.8,
 'BJS': 1.0,
 'J': 0.7,
 'JS': 0.8999999999999999,
 'S': 0.6}

In [254]:
a.get_disbelief()

{'B': 0.6,
 'BJ': 0.09999999999999998,
 'BS': 0.19999999999999996,
 'BJS': 0.0,
 'J': 0.30000000000000004,
 'JS': 0.10000000000000009,
 'S': 0.4}

In [255]:
a = MassFunction("./sample_data1/sample_data1.csv")
a.frame_of_discernment

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
JS,0.3
BS,0.1
BJS,0.1


In [256]:
a = MassFunction(df.to_dict()["Mass"])
a.frame_of_discernment

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
JS,0.3
BS,0.1
BJS,0.1


In [257]:
a = MassFunction(list(df.to_records()))
a.frame_of_discernment

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
JS,0.3
BS,0.1
BJS,0.1


In [258]:
a = MassFunction(np_array_input)
a.frame_of_discernment

,Mass
phi,0.0
B,0.1
J,0.2
S,0.1
BJ,0.1
JS,0.3
BS,0.1
BJS,0.1
